# Deep Learning Models etc

In [ ]:
from IPython.display import HTML
import import_ipynb
import random
import utils
utils.hide_toggle('Imports 1')

In [ ]:
import numpy as np
import torch
from torch import nn
from torch import optim
from IPython import display
utils.hide_toggle('Imports 2')

Compute accuracy of predictions.

In [ ]:
def accuracy(Net,X_test,y_test,verbose=True):
    Net.eval()
    m = X_test.shape[0]
    y_pred = Net(X_test)
    predicted = torch.max(y_pred, 1)[1]
    correct = (predicted == y_test).float().sum().item()
    if verbose: print(correct,m)
    accuracy = correct/m
    Net.train()
    return accuracy
utils.hide_toggle('Function: accuracy')

In [ ]:
def accuracy_variable(Net,data):
    step=0
    acc=0
    for (X,y) in data:
            y_pred = Net(X)
            step+=1
            acc+=accuracy(Net,X,y,verbose=False)
    a = acc/step
    return a

Generic training loop

In [ ]:
def Train(Net,data,epochs=20,lr=5e-2,Loss=nn.NLLLoss(),verbose=False):
    ### INSERT YOUR CODE HERE
    optimizer = Net.optimizer
    losses = []
    accs = []
    for epoch in range(epochs):
      correct = 0
      loss_ = 0.0
      length = 0
      for X_batch, y_batch in data:
        optimizer.zero_grad()
        predictions = Net(X_batch)
        pred = torch.max(predictions, 1)[1]
        loss = Loss(predictions, y_batch)
        loss.backward()
        optimizer.step()
        correct += (pred == y_batch).float().sum().item()
        loss_ += loss.item()
        length += X_batch.shape[0]
      acc = correct / length
      accs.append(acc)
      losses.append(loss.item())
      print('Epoch [{}/{}], Loss: {:.7f}, Accuracy: {:.4f}'.format(epoch+1, epochs, loss_/length, acc))

    return Net,losses,accs
utils.hide_toggle('Function Train')

Multi-layer perceptron with ReLU non-lineartities; for classification or regression.

In [ ]:
class MLP(nn.Module):
    def __init__(self,dims=[5,3,2],task='classification',lr=1e-3):
        super(MLP,self).__init__()
        self.dims=dims
        self.n = len(self.dims)-1
        self.task=task
        self.layers=nn.ModuleList()
        for i in range(self.n-1):
            self.layers.append(nn.Linear(dims[i],dims[i+1]))
            self.layers.append(nn.ReLU())
        if task=='classification': 
            self.layers.append(nn.Linear(dims[i+1],dims[i+2]))
            self.layers.append(nn.LogSoftmax(dim=1))
        elif task=='regression': 
            self.layers.append(nn.Linear(dims[i+1],dims[i+2]))
            self.layers.append(nn.Linear(dims[i+2],1))
        else: self.layers.append(nn.Linear(dims[i+1],dims[i+2]))
        self.optimizer = optim.Adam(self.parameters(),lr=lr)
    def forward(self,x):
        for l in self.layers:
            x = l(x)
        return(x)

Recurrent network using GRU

In [ ]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size,lr):
        # This just calls the base class constructor
        super().__init__()
        # Neural network layers assigned as attributes of a Module subclass
        # have their parameters registered for training automatically.
        self.input_size=input_size
        self.rnn = torch.nn.RNN(input_size, hidden_size, nonlinearity='relu', batch_first=True)
        self.linear = torch.nn.Linear(hidden_size, output_size)
        self.logsoft = nn.LogSoftmax(dim=-1)
        self.optimizer = optim.Adam(self.parameters(),lr=lr)
    def forward(self, x):
        # torch.nn.RNN also returns its hidden state but we don’t use it.
        # While the RNN can also take a hidden state as input, the RNN
        # gets passed a hidden state initialized with zeros by default.
        if self.input_size==1: x=x.unsqueeze(-1)
        ### INSERT YOUR CODE HERE
        hidden = torch.zeros(1, x.size(0), self.rnn.hidden_size)
        x, _ = self.rnn(x, hidden)
        x = self.linear(x)
        x = self.logsoft(x)
        x=x[:,-1,:]
        return x

In [ ]:
class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, lr):
        super().__init__()
        ### INSERT YOUR CODE HERE
        self.input_size=input_size
        self.hidden_size=hidden_size 
        self.lstm = torch.nn.LSTM(input_size, hidden_size, batch_first=True)
        self.linear = torch.nn.Linear(hidden_size, output_size)
        self.logsoft = nn.LogSoftmax(dim=-1)
        self.optimizer = optim.Adam(self.parameters(),lr=lr)
    def forward(self, x):
        if self.input_size==1: x=x.unsqueeze(-1)
        #### INSERT YOUR CODE HERE
        hidden = torch.zeros(1, x.size(0), self.hidden_size)
        cell = torch.zeros(1, x.size(0), self.hidden_size)
        x, _ = self.lstm(x, (hidden, cell))
        x = self.linear(x)
        x = self.logsoft(x)
        x=x[:,-1,:]
        return x

In [ ]:
import torch.nn.functional as F
class OneDimCNN(nn.Module):
    def __init__(self,input_size, hidden_size, output_size, lr):
        super().__init__()
        self.input_size=input_size
        self.hidden_size=hidden_size
        self.conv_1 = nn.Conv1d(in_channels=input_size, out_channels=hidden_size, kernel_size=5, stride=2)
        self.conv_2 = nn.Conv1d(in_channels=hidden_size, out_channels=output_size, kernel_size=1)
        self.pool = nn.AdaptiveAvgPool1d(25)
        self.linear = nn.Linear(25*output_size, output_size)
        self.logsoft = nn.LogSoftmax(dim=-1)
        self.optimizer = optim.Adam(self.parameters(),lr=lr)
    def forward(self, x):
        if self.input_size==1: x=x.unsqueeze(-1)
        x = x.transpose(1,2)
        x = self.conv_1(x)
        x = F.relu(x)
        x = self.conv_2(x)
        x = self.pool(x)
        x = x.flatten(1)
        x = self.linear(x)
        x = self.logsoft(x)
        return x

In [ ]:
class Transformer(nn.Module):
    def __init__(self, input_size, hidden_size, output_size,lr):
        # This just calls the base class constructor
        super().__init__()
        self.input_size = input_size
        self.d_model = 128
        self.linear_1 = nn.Linear(self.input_size, self.d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=self.d_model, nhead=8, dim_feedforward=hidden_size)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=1)
        self.relu = nn.ReLU()
        self.linear_2 = nn.Linear(self.d_model, output_size)
        self.logsoft = nn.LogSoftmax(dim=-1)
        self.optimizer = optim.Adam(self.parameters(),lr=lr)
    def forward(self, x):
        if self.input_size==1: x=x.unsqueeze(-1)
        x = x.permute(1, 0, 2)
        x = self.linear_1(x)
        x = self.relu(x)
        #Positional Encoding
        max_len = 25
        position = torch.arange(25).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, self.d_model, 2) * (-np.math.log(10000.0) / self.d_model))
        pe = torch.zeros(max_len, 1, self.d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        x = x + pe[:x.size(0)]
        x = self.transformer_encoder(x)
        x = x.transpose(1, 0)
        x = x.max(1)[0]
        x = self.relu(x)
        x = self.linear_2(x)
        x = self.logsoft(x)
        return x